## 01. Example of using the Calculate Mortage Payments using pywidget sliders

In [2]:
# ANSI escape codes for bold and color
bold = '\033[1m'
end_bold = '\033[0m'
red = '\033[91m'
green = '\033[92m'
yellow = '\033[93m'
blue = '\033[94m'
purple = '\033[95m'
cyan = '\033[96m'

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import matplotlib.pyplot as plt
from utils import calculate_mortgage_payments, calculate_cagr, super_balance
from datetime import datetime
# Create widgets for input parameters
house_price_widget = widgets.IntSlider(value=650000, min=500000, max=800000, step=10000, description='House Price:')
down_payment_widget = widgets.FloatSlider(value=20, min=5, max=30, step=1, description='% Down Payment:')
interest_rate_widget = widgets.FloatSlider(value=6.5, min=1, max=10, step=0.1, description='% Interest Rate:')
years_widget = widgets.IntSlider(value=30, min=1, max=30, step=1, description='Years:')
weekly_rent_widget = widgets.IntSlider(value=550, min=500, max=800, step=20, description='Initial Weekly Rent:')
# container = widgets.HBox(children=[house_price_widget, down_payment_widget, interest_rate_widget, years_widget, weekly_rent_widget])

def update_mortgage_payments(house_price, down_payment, interest_rate, loan_term_years, weekly_rent):
    water = 40
    insurance = 200
    council_tax = 140
    start_date = datetime(2024, 10, 1)  # Start date of loan
    df = calculate_mortgage_payments(down_payment, 
                                                   house_price, 
                                                   interest_rate, 
                                                   loan_term_years, 
                                                   start_date,
                                                   council_tax,
                                insurance,
                                water,
                                weekly_rent)
    (cash_after_tax, annualized_return, total_tax_paid, total_investment) = calculate_cagr(df, down_payment, house_price)
    initial_inv = df['InitialInvestments'].max()
    invest_while_rented = total_investment - initial_inv
    # print(f"{bold}{green}Monthly Payment: ${end_bold} ${df["Payment Amount"].max():.2f}")
    print(f"Monthly Payment: ${df['Payment Amount'].max():.2f}" )
    # print(f"Cash After Sale and Tax: ${cash_after_tax:.2f}" , f"Annualized Return: {annualized_return*100:.2f}%")
    # print(f"Initial Investments: ${initial_inv:.2f}", f"Net Investments while rented: ${invest_while_rented:.2f}",)
    metric_labels = ['Cash After Sale and Tax', 'Initial Investments', 'Net Investments while rented']
    metric_values = [round(cash_after_tax,0) , round(initial_inv,0), round(invest_while_rented,0)]
    # Create subplots using make_subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Net Rental Income', f'Total Gains Post Sale After {loan_term_years: .0f} years'))
    fig.add_trace(go.Bar(x=metric_labels,y =metric_values, hovertemplate='%{x}<br> $%{y}<extra></extra>', name='Total Gains'), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['Losses'].cumsum(), mode='lines', hovertemplate='Date: %{x}<br>Cumulative Net Income: %{y}<extra></extra>', name='Cumulative Net Income'), row=1, col=1)
    fig.update_layout(height=500, width=1000)
    fig.show()
    print(f"Total Investments: ${initial_inv+invest_while_rented:.2f}")
    print(f"Cash After Sale and Tax: ${cash_after_tax:.2f}") 
    print(f"Annualized Return: {annualized_return*100:.2f}%")
    

    


interact_widget = widgets.interact(update_mortgage_payments, 
                                   house_price=house_price_widget, 
                                   down_payment=down_payment_widget, 
                                   interest_rate=interest_rate_widget, 
                                   loan_term_years = years_widget, 
                                   weekly_rent=weekly_rent_widget)

display(interact_widget)





interactive(children=(IntSlider(value=650000, description='House Price:', max=800000, min=500000, step=10000),…

<function __main__.update_mortgage_payments(house_price, down_payment, interest_rate, loan_term_years, weekly_rent)>

In [4]:
def calculate_monthly_repayment(loan_amount, annual_interest_rate, loan_term_years):
    # Calculate monthly interest rate
    monthly_interest_rate = annual_interest_rate / 12 / 100

    # Calculate number of monthly payments
    num_monthly_payments = loan_term_years * 12

    # Calculate monthly repayment using the formula
    monthly_repayment = (loan_amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -num_monthly_payments)

    return monthly_repayment

# Example usage
loan_amount = 600000  # USD
annual_interest_rate = 6.84  # Annual interest rate (percentage)
loan_term_years = 30  # Loan term in years

monthly_repayment = calculate_monthly_repayment(loan_amount, annual_interest_rate, loan_term_years)
print(f"Monthly Repayment: ${monthly_repayment:.2f}")


Monthly Repayment: $3927.55


In [7]:
monthly_repayment = 3251
loan_amount = 227000
int_rate = .064
from utils import calculate_monthly_return_rate, calculate_daily_return_rate
d_rate= calculate_daily_return_rate(int_rate)
m_rate = calculate_monthly_return_rate(int_rate)
print(loan_amount*d_rate*31, loan_amount*m_rate)


1196.109987464085 1176.541324687635


In [29]:
from datetime import datetime, date, timedelta
import calendar
import pandas as pd
from utils import calculate_monthly_return_rate, calculate_daily_return_rate
def generate_end_of_month_dates_with_days(start_date, num_years, initial_loan_amount, interest_rate, monthly_repayment, rent, fee, monthly_expense):
    end_dates = []
    days_in_months = []
    loan_amounts=[]
    current_date = start_date
    loan_amount = initial_loan_amount
    prinicpals = []
    interests = []
    rental_incomes = []
    expenses = []
    cashflows = []
    # Generate end of month dates and days in each month for each year
    for _ in range(num_years):
        for month in range(1, 13):  # Loop through months
            
            end_date = current_date.replace(day=1) + timedelta(days=32)
            end_date = end_date.replace(day=1) - timedelta(days=1)  # Get last day of the month
            if month == 12:
                loan_amount = loan_amount - 20000
                rent = rent*1.05
                monthly_expense = monthly_expense*1.05
            elif month == 6:
                loan_amount = loan_amount - 30000
            # Calculate days in the month
            num_days_in_month = calendar.monthrange(end_date.year, end_date.month)[1]

            expense = calculate_expenses(rent, fee, monthly_expense)
            expenses.append(expense)
            rental_incomes.append(rent)
            end_dates.append(end_date)
            days_in_months.append(num_days_in_month)
            d_rate = calculate_daily_return_rate(interest_rate/100)
            interest = d_rate*loan_amount*num_days_in_month
            principal = monthly_repayment - interest
            loan_amount = loan_amount - principal
            loan_amounts.append(loan_amount)
            prinicpals.append(principal)
            interests.append(interest)
            current_date = end_date + timedelta(days=1)  # Move to next month
            cashflow = rent - expense - monthly_repayment
            cashflows.append(cashflow)

    payment_df = pd.DataFrame({
        'Date': end_dates,
        'Payment Amount': monthly_repayment,
        'Principal': prinicpals,
        'Interest': interests,
        'LoanAmount': loan_amounts,
        'Expenses': expenses,
        'Income': rental_incomes,
        'Cashflow': cashflows
    })

    return payment_df

def calculate_expenses(rent, fee, monthly_expense):
    return ((fee/100)*rent)+monthly_expense
# Example usage
start_date = date(2024, 4, 30)
num_years = 3
monthly_repayment = 3251
loan_amount = 227000
int_rate = 6.04
rent = (690*52)/12
rent_increase = 5.0
water = 50
insurance = 200
council = 210
fee=8.25
monthly_expense = water + insurance + council


df = generate_end_of_month_dates_with_days(start_date, num_years, loan_amount,int_rate, monthly_repayment, rent, fee, monthly_expense)
df
# for date, days in zip(end_of_month_dates, days_in_months):
#     print(date, days)


,Date,Payment Amount,Principal,Interest,LoanAmount,Expenses,Income,Cashflow
0,2024-04-30,3251,2156.718691,1094.281309,224843.281309,706.675000,2990.00000,-967.675000
1,2024-05-31,3251,2130.985932,1120.014068,222712.295377,706.675000,2990.00000,-967.675000
2,2024-06-30,3251,2177.388096,1073.611904,220534.907281,706.675000,2990.00000,-967.675000
3,2024-07-31,3251,2152.447278,1098.552722,218382.460003,706.675000,2990.00000,-967.675000
4,2024-08-31,3251,2163.169285,1087.830715,216219.290718,706.675000,2990.00000,-967.675000
5,2024-09-30,3251,2353.307096,897.692904,183865.983622,706.675000,2990.00000,-967.675000
6,2024-10-31,3251,2335.106551,915.893449,181530.877071,706.675000,2990.00000,-967.675000
7,2024-11-30,3251,2375.908168,875.091832,179154.968902,706.675000,2990.00000,-967.675000
8,2024-12-31,3251,2358.573576,892.426424,176796.395326,706.675000,2990.00000,-967.675000
9,2025-01-31,3251,2370.322362,880.677638,174426.072965,706.675000,2990.00000,-967.675000


In [8]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the webpage
rent_url = 'https://sqmresearch.com.au/weekly-rents.php?postcode=4509&t=1'
response = requests.get(rent_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table containing the rent data
    table = soup.find('div', class_='changetable')
    data_text = []
    if table:
        # Extract rows from the table
        rows = table.find_all('tr')
        
        # Assume the first row contains headers and skip it
        for row in rows[1:]:
            # Extract columns from each row
            columns = row.find_all('td')
            for col in columns:
                data = col.get_text(strip=True)
                data_text.append(data)
            
else:
    print("Error fetching the webpage.")



In [23]:
annual_change_rent = data_text[-1]
current_weekly_rents = float(data_text[-8])

In [24]:
current_weekly_rents

629.44

In [26]:
base_url='https://sqmresearch.com.au'
rent_url = base_url+'/weekly-rents.php?postcode=4509&t=1'
rent_url


'https://sqmresearch.com.au/weekly-rents.php?postcode=4509&t=1'

In [1]:
from utils import get_postcode_data
x= get_postcode_data(9999)
x['rent'][0]

In [2]:
x['rent'][1]